In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
#Modify DATA_PATH if needed
DATA_TRAIN_PATH = '../../data_project1/train.csv'
y, tX_old, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
import csv

def load_csv_data(data_path, sub_sample=False):
    """Loads data and returns y (class labels), tX (features) and ids (event ids)"""
    y = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    x = np.genfromtxt(data_path, delimiter=",", skip_header=1)
    ids = x[:, 0].astype(np.int)
    input_data = x[:, 2:]

    # convert class labels from strings to binary (-1,1)
    yb = np.ones(len(y))
    yb[np.where(y=='b')] = -1
    
    # sub-sample
    if sub_sample:
        yb = yb[::50]
        input_data = input_data[::50]
        ids = ids[::50]

    return yb, input_data, ids


def predict_labels(weights, data):
    """Generates class predictions given weights, and a test data matrix"""
    y_pred = np.dot(data, weights)
    y_pred[np.where(y_pred <= 0)] = -1
    y_pred[np.where(y_pred > 0)] = 1
    
    return y_pred


def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})
            
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

def standardize(x):
    """Standardize the original data set."""
    mean_x = np.mean(x, axis=0)
    x = x - mean_x
    std_x = np.std(x, axis=0)
    x = x / std_x
    return x, mean_x, std_x

In [4]:
print("Training examples: ", tX_old, " & shape: ")
print("Targets: ", y)
print("Ids: ",ids)
print("Shapes of tX, y & Ids: ", tX_old.shape, y.shape, ids.shape)

Training examples:  [[ 138.47    51.655   97.827 ...    1.24    -2.475  113.497]
 [ 160.937   68.768  103.235 ... -999.    -999.      46.226]
 [-999.     162.172  125.953 ... -999.    -999.      44.251]
 ...
 [ 105.457   60.526   75.839 ... -999.    -999.      41.992]
 [  94.951   19.362   68.812 ... -999.    -999.       0.   ]
 [-999.      72.756   70.831 ... -999.    -999.       0.   ]]  & shape: 
Targets:  [ 1. -1. -1. ...  1. -1. -1.]
Ids:  [100000 100001 100002 ... 349997 349998 349999]
Shapes of tX, y & Ids:  (250000, 30) (250000,) (250000,)


In [5]:
'''PREPROCESS: FEATURE EXTRACTION AND STANDARDIZE'''
#Feature extraction
tX = np.hstack((tX_old[:,1:4], tX_old[:,7:12]))
tX = np.hstack((tX, tX_old[:,13:23]))
#Standardize
tX, tX_mean, tX_std = standardize(tX)

In [6]:
'''SPLIT INTO TRAIN AND VALIDATION'''

train_valid_split = int(tX.shape[0] / 10)
print("Validation data size: ", train_valid_split)
tX_valid = tX[train_valid_split:,:]
y_valid = y[train_valid_split:]
id_valid = ids[train_valid_split:]

tX = tX[:train_valid_split]
y = y[:train_valid_split]
ids = ids[:train_valid_split]

print("Shapes of tX, y & Ids for Training: ", tX.shape, y.shape, ids.shape)
print("Shapes of tX, y & Ids for Validation: ", tX_valid.shape, y_valid.shape, id_valid.shape)

Validation data size:  25000
Shapes of tX, y & Ids for Training:  (25000, 18) (25000,) (25000,)
Shapes of tX, y & Ids for Validation:  (225000, 18) (225000,) (225000,)


In [7]:
'''CHECK MODEL SUCCESS: DELETE BEFORE SUBMISSION'''
from sklearn import linear_model
from sklearn import metrics
clf = linear_model.SGDClassifier(max_iter=500, tol=1e-3)
clf.fit(tX, y)
print(metrics.accuracy_score(clf.predict(tX_valid), y_valid))

0.7360888888888889


## Do your thing crazy machine learning thing here :) ...

In [32]:
'''GRAD AND LOSS FUNCTIONS'''
def compute_loss(y, tx, w, typ):
    '''typ = <LOSS_TYPE(WITH CAPITAL LETTERS)>'''
    loss = 0
    N = y.shape[0]
    if typ == "MSE":
        loss = (1/(2*N))*np.sum(np.square(y - (tx@w)))        
    elif typ == "MAE":
        loss = (1/(2*N))*np.sum(np.abs(y - (tx@w)))
    return loss

def compute_gradient(y, tx, w):
    '''GRADIENT COMPUTATION'''
    N = y.shape[0]
    e = y - tx@w
    grad = (-1/N) * (tx.T@e)
    return grad

def compute_stoch_gradient(y, tx, w):
    '''STOCHASTIC GRADIENT DESCENT GRADIENT COMPUTATION''' 
    N = y.shape[0]
    e = y - tx@w
    grad = (-1/N)*(tx.T@e)
    return grad

def compute_ls_loss(y, tx, w):
    '''LEAST SQUARES WITH NORMAL EQUATIONS LOSS COMPUTATION'''
    loss = 0
    N = y.shape[0]
    loss = (1/(2*N))*(tx.T@(y - tX@w))
    
def compute_rdg_loss(y, tx, w, lambda_):
    '''RIDGE REGRESSION LOSS COMPUTATION'''
    loss = 0
    N = y.shape[0]
    loss = (1/(2*N))*np.sum(np.square(y - (tx@w))) + (lambda_*np.sum(np.square(w)))
    return loss

def sigmoid(tx, w):
    '''SIGMOID CALCULATION'''
    return 1 / (1 + np.exp(-1*(tx@w)))

def compute_log_loss(y, tx, w):
    '''LOGISTIC LOSS'''
    loss = 0
    sigm = sigmoid(tx,w)
    N = y.shape[0]
    loss = (-1/N)*(np.sum(np.log(sigm).T * -y - (np.log(1 - sigm).T * (1-y))))
    return loss

In [33]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    '''BATCH GRADIENT DESCENT'''
    w = initial_w
    for n_iter in range(max_iters):
        loss = compute_loss(y, tx, w, "MSE")
        grad = compute_gradient(y, tx, w)
        w = w - (gamma * grad)
        print("Gradient Descent({bi}/{ti}): loss={l}".format(
              bi=n_iter, ti=max_iters - 1, l=loss))

    return (w, loss)

In [34]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    '''STOCHASTIC GRADIENT DESCENT'''
    w = initial_w 
    for n_iter in range(max_iters):
        for minibatch_y, minibatch_tx in batch_iter(y, tx, 1):
            loss = compute_loss(minibatch_y, minibatch_tx, w, "MSE")
            grad = compute_gradient(minibatch_y, minibatch_tx, w)
            w = w - gamma * grad
            print("Stochastic Gradient Descent({bi}/{ti}): loss={l}".format(
              bi=n_iter, ti=max_iters - 1, l=loss))
    return (w, loss)

In [41]:
def least_squares(y, tx):
    '''COMPUTE W_STAR: WEIGHT FOR NORMAL EQUATIONS BY LINEAR EQUATION SOLVER'''
    w_star = np.linalg.solve(tx.T@tx, tx.T@y)
    loss = compute_ls_loss(y, tx, w_star)
    return (w_star,loss)

In [38]:
def ridge_regression(y, tx, lambda_):
    '''RIDGE REGRESSION WITH LAMBDA PARAMETER AS REGULARIZATION PARAMETER'''
    N = y.shape[0]
    w_ridge = np.linalg.solve((tx.T@tx)+(lambda_/(2*N))*np.identity(tx.shape[1]), tx.T@y)
    loss = compute_rdg_loss(y, tx, w_ridge, lambda_)
    return (w_ridge, loss)
    

In [13]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    '''FOR GRADIENT DESCENT'''
    w = initial_w
    for n_iter in range(max_iters):
        loss = compute_log_loss(y, tx, w)
        grad = compute_gradient(y, tx, w)
        w = w - (gamma * grad)
        print("Logistic Regression Gradient Descent({bi}/{ti}): loss={l}".format(
              bi=n_iter, ti=max_iters - 1, l=loss))

    return (w, loss)
    '''FOR STOCHASTIC GRADIENT DESCENT'''
    '''
    w = initial_w 
    lr_scheduler = [(max_iters) / 4, (2* max_iters) / 4,(3* max_iters) / 4]
    for n_iter in range(max_iters):
        if n_iter in lr_scheduler:
            gamma = gamma / 10
        for minibatch_y, minibatch_tx in batch_iter(y, tx, 1):
            loss = compute_loss(minibatch_y, minibatch_tx, w, "MSE")
            grad = compute_gradient(minibatch_y, minibatch_tx, w)
            w = w - gamma * grad
            print("Stochastic Gradient Descent({bi}/{ti}): loss={l}".format(
              bi=n_iter, ti=max_iters - 1, l=loss))
    return (w, loss)
    '''

## Generate predictions and save ouput in csv format for submission:

In [14]:
DATA_TEST_PATH = '../../data_project1/test.csv' # TODO: download train data and supply path here 
_, tX_test_old, ids_test = load_csv_data(DATA_TEST_PATH)

In [15]:
'''PREPROCESS: FEATURE EXTRACTION AND STANDARDIZE FOR TEST'''
#Feature extraction
tX_test = np.hstack((tX_test_old[:,1:4], tX_test_old[:,7:12]))
tX_test = np.hstack((tX_test, tX_test_old[:,13:23]))
#Standardize
tX_test, tX_test_mean, tX_test_std = standardize(tX_test)

print("Shapes of tX & Ids for Testing: ", tX_test.shape, ids.shape)

Shapes of tX & Ids for Testing:  (568238, 18) (25000,)


In [16]:
ww = np.random.rand(tX.shape[1])
init_w = np.array(ww, dtype=np.float64)
#init_w = np.zeros(tX.shape[1])
print(init_w.shape)
print(init_w)

(18,)
[0.19199625 0.37676237 0.39117816 0.32511406 0.19301523 0.05617632
 0.11564319 0.69647416 0.767847   0.65848925 0.71485873 0.39977526
 0.41875458 0.0414966  0.82033364 0.04348927 0.68064777 0.83111796]


In [28]:
'''HYPER PARAMETERS FOR TUNING'''
max_iter = 500
alpha = 1e-2

In [29]:
'''PREDICTIONS FOR MODELS'''
'''BATCH GD'''
(w1,loss1) = least_squares_GD(y, tX, init_w, max_iter, alpha)
gd_tr_pred = predict_labels(w1, tX_valid)
print((gd_tr_pred == y_valid).mean())
gd_pred = predict_labels(w1, tX_test)

Gradient Descent(0/499): loss=5.93796690085545
Gradient Descent(1/499): loss=5.488485544046356
Gradient Descent(2/499): loss=5.080486109511487
Gradient Descent(3/499): loss=4.709989595829887
Gradient Descent(4/499): loss=4.37340344398495
Gradient Descent(5/499): loss=4.0674838460652
Gradient Descent(6/499): loss=3.789301735791361
Gradient Descent(7/499): loss=3.5362121010068757
Gradient Descent(8/499): loss=3.305826293450161
Gradient Descent(9/499): loss=3.095987042868281
Gradient Descent(10/499): loss=2.9047459111697096
Gradient Descent(11/499): loss=2.7303429481519164
Gradient Descent(12/499): loss=2.571188333651333
Gradient Descent(13/499): loss=2.4258458119959476
Gradient Descent(14/499): loss=2.2930177436170442
Gradient Descent(15/499): loss=2.171531615797656
Gradient Descent(16/499): loss=2.0603278699823973
Gradient Descent(17/499): loss=1.9584489170102783
Gradient Descent(18/499): loss=1.865029224206388
Gradient Descent(19/499): loss=1.779286369613382
Gradient Descent(20/499): l

Gradient Descent(320/499): loss=0.419923600534203
Gradient Descent(321/499): loss=0.41983672725772875
Gradient Descent(322/499): loss=0.41975068591217224
Gradient Descent(323/499): loss=0.4196654660356852
Gradient Descent(324/499): loss=0.41958105732065026
Gradient Descent(325/499): loss=0.41949744961099183
Gradient Descent(326/499): loss=0.419414632899543
Gradient Descent(327/499): loss=0.41933259732546635
Gradient Descent(328/499): loss=0.41925133317172875
Gradient Descent(329/499): loss=0.4191708308626262
Gradient Descent(330/499): loss=0.4190910809613614
Gradient Descent(331/499): loss=0.4190120741676689
Gradient Descent(332/499): loss=0.4189338013154894
Gradient Descent(333/499): loss=0.4188562533706912
Gradient Descent(334/499): loss=0.41877942142883645
Gradient Descent(335/499): loss=0.41870329671299356
Gradient Descent(336/499): loss=0.4186278705715933
Gradient Descent(337/499): loss=0.41855313447632664
Gradient Descent(338/499): loss=0.41847908002008544
Gradient Descent(339/49

In [27]:
'''SGD'''
(w2,loss2) = least_squares_SGD(y, tX, init_w, max_iter, alpha)
sgd_tr_pred = predict_labels(w2, tX_valid)
print((sgd_tr_pred == y_valid).mean())
sgd_pred = predict_labels(w2, tX_test)

Stochastic Gradient Descent(0/499): loss=0.019737475721124543
Stochastic Gradient Descent(1/499): loss=8.74648505546766
Stochastic Gradient Descent(2/499): loss=2.6693925002566634
Stochastic Gradient Descent(3/499): loss=0.028992837559970387
Stochastic Gradient Descent(4/499): loss=0.45982636084022105
Stochastic Gradient Descent(5/499): loss=2.066410350005182
Stochastic Gradient Descent(6/499): loss=10.770847131757902
Stochastic Gradient Descent(7/499): loss=12.39264073947729
Stochastic Gradient Descent(8/499): loss=0.14530440635441372
Stochastic Gradient Descent(9/499): loss=2.751803002495881
Stochastic Gradient Descent(10/499): loss=13.66593493946173
Stochastic Gradient Descent(11/499): loss=3.2617310943876383
Stochastic Gradient Descent(12/499): loss=0.030252668998152623
Stochastic Gradient Descent(13/499): loss=4.999194817489299
Stochastic Gradient Descent(14/499): loss=0.13994436000579077
Stochastic Gradient Descent(15/499): loss=3.4413051919807707
Stochastic Gradient Descent(16/4

Stochastic Gradient Descent(166/499): loss=1.0109907973807157
Stochastic Gradient Descent(167/499): loss=0.5897399044829541
Stochastic Gradient Descent(168/499): loss=0.13889494421195642
Stochastic Gradient Descent(169/499): loss=0.34864809706348043
Stochastic Gradient Descent(170/499): loss=5.242197448136968
Stochastic Gradient Descent(171/499): loss=2.6217487442953926
Stochastic Gradient Descent(172/499): loss=0.002760883796889363
Stochastic Gradient Descent(173/499): loss=5.215890330798556
Stochastic Gradient Descent(174/499): loss=0.59195000895803
Stochastic Gradient Descent(175/499): loss=14.223422146424731
Stochastic Gradient Descent(176/499): loss=26.32705046642058
Stochastic Gradient Descent(177/499): loss=7.455334880905274
Stochastic Gradient Descent(178/499): loss=0.3187295972104611
Stochastic Gradient Descent(179/499): loss=16.047046971218744
Stochastic Gradient Descent(180/499): loss=5.373981517366374
Stochastic Gradient Descent(181/499): loss=0.7623450535595617
Stochastic 

Stochastic Gradient Descent(344/499): loss=2.261993197280613
Stochastic Gradient Descent(345/499): loss=0.3353253223166464
Stochastic Gradient Descent(346/499): loss=0.0001248982942883417
Stochastic Gradient Descent(347/499): loss=2.5565865079355063
Stochastic Gradient Descent(348/499): loss=3.4057711305540623
Stochastic Gradient Descent(349/499): loss=0.8574153681768067
Stochastic Gradient Descent(350/499): loss=83.49935179961429
Stochastic Gradient Descent(351/499): loss=1.3381399222226558
Stochastic Gradient Descent(352/499): loss=0.2729691800371401
Stochastic Gradient Descent(353/499): loss=0.012337408107425816
Stochastic Gradient Descent(354/499): loss=0.17233953182391393
Stochastic Gradient Descent(355/499): loss=0.8226091188467377
Stochastic Gradient Descent(356/499): loss=13.424511072498474
Stochastic Gradient Descent(357/499): loss=18.664356266459315
Stochastic Gradient Descent(358/499): loss=0.12420617276476896
Stochastic Gradient Descent(359/499): loss=5.703835488479696
Stoc

In [43]:
'''LS WITH NORMAL EQ'''
(w3,loss3) = least_squares(y, tX)
ls_tr_pred = predict_labels(w3, tX_valid)
print((ls_tr_pred == y_valid).mean())
ls_pred = predict_labels(w3, tX_test)

0.7066711111111111


In [44]:
'''RIDGE REGRESSION'''
'''CHOOSE BEST LAMBDA'''
lambda_ = np.logspace(-1, -6, 30)
min_loss = 1000000
ind = 0
for i in range(lambda_.shape[0]):
    (w4,loss4) = ridge_regression(y, tX, lambda_[i])
    if min_loss > loss4:
        min_loss = loss4
        ind = i
(w4,loss4) = ridge_regression(y, tX, lambda_[ind])       
rd_tr_pred = predict_labels(w4, tX_valid)
print((rd_tr_pred == y_valid).mean())
rd_pred = predict_labels(w4, tX_test)

0.7066711111111111


In [ ]:
'''LOGISTIC REGRESSION'''
(w5,loss5) = logistic_regression(y, tX, init_w, max_iter, alpha)
log_tr_pred = predict_labels(w5, tX_valid)
print((log_tr_pred == y_valid).mean())
log_pred = predict_labels(w5, tX_test)

In [ ]:
OUTPUT_PATH = '../../data_project1/results_least_gd.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w1, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)